<a href="https://colab.research.google.com/github/GGpark1/Deep_Learning_Study/blob/master/Word2Vec_with_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#gensim 패키지 import

!pip install gensim --upgrade

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [5]:
import gensim
import gensim.downloader as api

gensim.__version__
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [7]:
#gensim에 학습된 단어 확인

for idx, word in enumerate(wv.index_to_key):
    if idx == 10:
        break

    print(f"word #{idx}/{len(wv.index_to_key)} is '{word}'")

word #0/3000000 is '</s>'
word #1/3000000 is 'in'
word #2/3000000 is 'for'
word #3/3000000 is 'that'
word #4/3000000 is 'is'
word #5/3000000 is 'on'
word #6/3000000 is '##'
word #7/3000000 is 'The'
word #8/3000000 is 'with'
word #9/3000000 is 'said'


In [8]:
#벡터화된 단어 확인

vec_king = wv['king']

print(f"임베딩 벡터의 차원 수 : {vec_king.shape}\n")
print(f"'king'의 임베딩 벡터 \n\n {vec_king}")

임베딩 벡터의 차원 수 : (300,)

'king'의 임베딩 벡터 

 [ 1.25976562e-01  2.97851562e-02  8.60595703e-03  1.39648438e-01
 -2.56347656e-02 -3.61328125e-02  1.11816406e-01 -1.98242188e-01
  5.12695312e-02  3.63281250e-01 -2.42187500e-01 -3.02734375e-01
 -1.77734375e-01 -2.49023438e-02 -1.67968750e-01 -1.69921875e-01
  3.46679688e-02  5.21850586e-03  4.63867188e-02  1.28906250e-01
  1.36718750e-01  1.12792969e-01  5.95703125e-02  1.36718750e-01
  1.01074219e-01 -1.76757812e-01 -2.51953125e-01  5.98144531e-02
  3.41796875e-01 -3.11279297e-02  1.04492188e-01  6.17675781e-02
  1.24511719e-01  4.00390625e-01 -3.22265625e-01  8.39843750e-02
  3.90625000e-02  5.85937500e-03  7.03125000e-02  1.72851562e-01
  1.38671875e-01 -2.31445312e-01  2.83203125e-01  1.42578125e-01
  3.41796875e-01 -2.39257812e-02 -1.09863281e-01  3.32031250e-02
 -5.46875000e-02  1.53198242e-02 -1.62109375e-01  1.58203125e-01
 -2.59765625e-01  2.01416016e-02 -1.63085938e-01  1.35803223e-03
 -1.44531250e-01 -5.68847656e-02  4.29687500e-02 

In [9]:
#oov 확인

unk = 'cameroon'

try:
    vec_unk = wv[unk]
except KeyError:
    print(f"""단어 "{unk}"은 해당 모델에는 등장하지 않는 단어입니다.""" )

단어 "cameroon"은 해당 모델에는 등장하지 않는 단어입니다.


In [10]:
pairs = [
         ('car', 'minivan'),
         ('car', 'bicycle'),
         ('car', 'airplane'),
         ('car', 'cereal'),
         ('car', 'democracy')]

for w1, w2 in pairs:
    print(f'{w1} ==== {w2}\t {wv.similarity(w1, w2):.2f}')

car ==== minivan	 0.69
car ==== bicycle	 0.54
car ==== airplane	 0.42
car ==== cereal	 0.14
car ==== democracy	 0.08


In [13]:
for i, (word, similarity) in enumerate(wv.most_similar(positive=['car', 'minivan'], topn=6)):
    print(f"Top {i+1} : {word}, {similarity}")

Top 1 : SUV, 0.8532192707061768
Top 2 : vehicle, 0.8175783753395081
Top 3 : pickup_truck, 0.7763688564300537
Top 4 : Jeep, 0.7567334175109863
Top 5 : Ford_Explorer, 0.7565720081329346
Top 6 : sedan, 0.7446292042732239


In [14]:
#king 벡터에 women 벡터를 더한 뒤(positive) men 벡터를 빼주는(negative) 과정
#walking 벡터에 swam 벡터를 더한 뒤 walked 벡터를 빼주는 과정

print(wv.most_similar(positive=['king', 'women'], negative=['men'], topn=1))
print(wv.most_similar(positive=['walking', 'swam'], negative=['walked'], topn=1))

[('queen', 0.6525818109512329)]
[('swimming', 0.7448815703392029)]


In [15]:
#단어 list 중에서 가장 관계 없는 단어 뽑아보기

print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


In [16]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb

In [17]:
#임베딩 벡터를 사용하여 문장 분류하기
#seed 고정
#데이터셋 split

tf.random.set_seed(42)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)

17473536/17464789 [==============================] - 1s 0us/step


In [18]:
print(f"Train set shape : {X_train.shape}")
print(f"Test set shape : {X_test.shape}")

Train set shape : (25000,)
Test set shape : (25000,)


In [35]:
#index를 text로 변환하고 문장으로 만드는 함수

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i) for i in text])

In [38]:
decode_review(X_train[0])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit the

In [41]:
#텍스트 학습

sentences = [decode_review(idx) for idx in X_train]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [44]:
#임베딩 매트릭스를 만들기 위한 단어 집합 크기 계산
vocab_size = len(tokenizer.word_index) + 1
vocab_size

19999

In [ ]:
#text를 벡터화

X_encoded = tokenizer.texts_to_sequences(sentences)
X_encoded

In [47]:
print(f'학습 데이터에 있는 문서의 평균 토큰 수: {np.mean([len(sent) for sent in X_encoded], dtype=int)}')

학습 데이터에 있는 문서의 평균 토큰 수: 238


In [48]:
#벡터를 패딩처리

maxlen_pad = 400 

X_train=pad_sequences(X_encoded, maxlen=maxlen_pad, padding = 'post')
y_train=np.array(y_train)

In [52]:
#임베딩 가중치 행렬 만들기

embedding_matrix = np.zeros((vocab_size, 300))
def get_vector(word):
    if word in wv:
        return wv[word]
    else:
        return None

for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [53]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.09082031, -0.21777344, -0.07226562, ...,  0.32226562,
        -0.05957031, -0.11328125],
       [-0.265625  ,  0.25585938, -0.01855469, ...,  0.0480957 ,
         0.34765625,  0.01177979],
       [-0.05541992,  0.26367188,  0.18457031, ..., -0.16503906,
        -0.25976562, -0.14257812]])

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [55]:
#모델 구축

model = Sequential()
model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = maxlen_pad, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation = 'sigmoid'))

In [56]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
313/313 [==============================] - 13s 37ms/step - loss: 0.6924 - acc: 0.5268 - val_loss: 0.6907 - val_acc: 0.5996
Epoch 2/20
313/313 [==============================] - 11s 34ms/step - loss: 0.6901 - acc: 0.5767 - val_loss: 0.6882 - val_acc: 0.5972
Epoch 3/20
313/313 [==============================] - 11s 34ms/step - loss: 0.6880 - acc: 0.5899 - val_loss: 0.6859 - val_acc: 0.6006
Epoch 4/20
313/313 [==============================] - 11s 34ms/step - loss: 0.6861 - acc: 0.5918 - val_loss: 0.6837 - val_acc: 0.5928
Epoch 5/20
313/313 [==============================] - 11s 36ms/step - loss: 0.6845 - acc: 0.5949 - val_loss: 0.6824 - val_acc: 0.5970
Epoch 6/20
313/313 [==============================] - 11s 34ms/step - loss: 0.6827 - acc: 0.5957 - val_loss: 0.6800 - val_acc: 0.5946
Epoch 7/20
313/313 [==============================] - 11s 34ms/step - loss: 0.6813 - acc: 0.6028 - val_loss: 0.6787 - val_acc: 0.6112
Epoch 8/20
313/313 [==============================] - 11s 34ms

In [60]:
test_sentences = [decode_review(idx) for idx in X_test]

X_test_encoded = tokenizer.texts_to_sequences(test_sentences)

X_test=pad_sequences(X_test_encoded, maxlen=400, padding='post')
y_test=np.array(y_test)

In [61]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 6s 8ms/step - loss: 0.6680 - acc: 0.6093


[0.6679654121398926, 0.609279990196228]